In [31]:
import requests
import pandas as pd
from pprint import pprint
import json
import datetime

In [5]:
team1 = 'Manchester United'
team2 = 'Borussia Dortmund'

In [6]:
header_params = {
            'x-rapidapi-key': "848eec715emsh5f15e8bc1776ee0p1826d1jsn93d8284e8db5",
            'x-rapidapi-host': "api-football-v1.p.rapidapi.com"
            }

In [9]:
def get_team_id(team):    
    url = "https://api-football-v1.p.rapidapi.com/v3/teams"

    querystring = {"name":team}

    response = requests.request("GET", url, headers=header_params, params=querystring)

    data = json.loads(response.text)
    
    return data['response'][0]['team']['id']

In [10]:
team_id_1 = get_team_id(team1)
team_id_2 = get_team_id(team2)

In [38]:
url = "https://api-football-v1.p.rapidapi.com/v3/standings"

querystring = {"season":"2020","team":"33"}

response = requests.request("GET", url, headers=header_params, params=querystring)

print(response.text)

{"get":"standings","parameters":{"season":"2020","team":"33"},"errors":[],"results":2,"paging":{"current":1,"total":1},"response":[{"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\/\/media.api-sports.io\/football\/leagues\/39.png","flag":"https:\/\/media.api-sports.io\/flags\/gb.svg","season":2020,"standings":[[{"rank":2,"team":{"id":33,"name":"Manchester United","logo":"https:\/\/media.api-sports.io\/football\/teams\/33.png"},"points":74,"goalsDiff":29,"group":"Premier League","form":"WDLLW","status":"same","description":"Promotion - Champions League (Group Stage)","all":{"played":38,"win":21,"draw":11,"lose":6,"goals":{"for":73,"against":44}},"home":{"played":19,"win":9,"draw":4,"lose":6,"goals":{"for":38,"against":28}},"away":{"played":19,"win":12,"draw":7,"lose":0,"goals":{"for":35,"against":16}},"update":"2021-05-23T00:00:00+00:00"}]]}},{"league":{"id":2,"name":"UEFA Champions League","country":"World","logo":"https:\/\/media.api-sports.io\/football\/l

In [27]:
def get_standings(team_id):
    url = "https://api-football-v1.p.rapidapi.com/v3/standings"
    
    year = datetime.datetime.now().year

    querystring = {"season":str(year-1),"team":str(team_id)}

    response = requests.request("GET", url, headers=header_params, params=querystring)
    
    data = json.loads(response.text)
    
    return data['response'][0]['league']['standings'][0][0]['rank']

In [28]:
get_standings(team_id_1)

2

In [54]:
def get_h2h(team_id_1, team_id_2):    
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures/headtohead"
    
    team_id_1 = get_team_id(team1)
    team_id_2 = get_team_id(team2)
    
    h2h_string = str(team_id_1)+'-'+str(team_id_2)

    querystring = {"h2h":h2h_string}

    headers = header_params

    response = requests.request("GET", url, headers=headers, params=querystring)
    data = json.loads(response.text)

    df = pd.DataFrame([d['teams']['home']['name'], 
                       d['teams']['away']['name'],
                       d['fixture']['date'],
                       d['goals']['home'],
                       d['goals']['away']] for d in data['response'])
    df = df.rename(columns={0:'home_team', 1:'away_team', 2:'date', 3:'home_goals', 4:'away_goals'})
    
    return df

In [55]:
get_h2h(team1, team2)

,home_team,away_team,date,home_goals,away_goals
0,Manchester United,Borussia Dortmund,2016-07-22T12:00:00+00:00,1,4


In [75]:
def get_injured_player(fixture_id):
    url = "https://api-football-v1.p.rapidapi.com/v3/injuries"

    querystring = {'fixture':str(fixture_id)}

    headers = header_params

    response = requests.request("GET", url, headers=headers, params=querystring)
    data = json.loads(response.text)

    df = pd.DataFrame([d['player']['id'], d['player_name']['name']] for d in data['response'])
    df = df.rename(columns={0:'player_id', 1:'player'})

    return df

In [93]:
def get_starting_11(team_id, fixture_id):
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures/lineups"

    querystring = {"team":str(team_id), 'fixture':str(fixture_id)}

    headers = header_params

    response = requests.request("GET", url, headers=headers, params=querystring)
    data = json.loads(response.text)
    
    starting_player_ids = [p['player']['id'] for p in data['response'][0]['startXI']]

    return starting_player_ids

In [90]:
get_starting_11(463, 215662)

[6258, 6261, 52701, 6268, 6262, 6474, 6269, 6212, 6093, 6126, 6561]

In [105]:
def get_last_10_fixture_ids(team_id):
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"

    querystring = {"last":"10", 'team':str(team_id)}

    headers = header_params

    response = requests.request("GET", url, headers=headers, params=querystring)
    data = json.loads(response.text)
    
    fixture_ids = [f['fixture']['id'] for f in data['response']]

    return fixture_ids

In [106]:
get_last_10_fixture_ids(team_id_1)

[702510,
 592875,
 592861,
 592831,
 592849,
 592837,
 698712,
 698711,
 592819,
 592812]

In [98]:
def get_next_fixture_id(team_id):
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"

    querystring = {"next":"1", 'team':str(team_id)}

    headers = header_params

    response = requests.request("GET", url, headers=headers, params=querystring)
    data = json.loads(response.text)  
    
    return data['response'][0]['fixture']['id']

In [99]:
get_next_fixture_id(team_id_1)

IndexError: list index out of range

In [110]:
def num_important_injured_players(team_id):
    try:
        num_important_injured_players = 0
        upcoming_fixture_id = get_next_fixture_id(team_id)
        df = get_injured_player(upcoming_fixture_id)
        if len(df['player_id'])==0:
            print('No players injured')
        else:
            last_10_fixture_ids = get_last_10_fixture_ids(team_id)
            important_players = []
            for fixture in last_10_fixture_ids:
                starting_11 = get_starting_11(fixture, team_id)
                for player in starting_11:
                    important_players.append(player)
            for player in df['player_id']:
                if player in important_players:
                    num_important_injured_players += 1
        return num_important_injured_players
    except:
        print('There is no upcoming reference game available')

In [112]:
for i in range(100,150):
    num_important_injured_players(i)

There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference game available
There is no upcoming reference gam